In [9]:
import sys

sys.path.append("/Users/ansel/ai/podcast-pipeline")
from pathlib import Path

from chonkie import LateChunker

from app.utils import merge_srt

chunker = LateChunker(
    embedding_model="all-MiniLM-L6-v2",
    mode="sentence",
    chunk_size=512,
    min_sentences_per_chunk=1,
    min_characters_per_sentence=12,
    delim="\n",
)


def chunks_to_text(chunks):
    chunk_text = [chunk.text for chunk in chunks]
    return "\n".join(chunk_text)


def srt_to_text(file_path):
    transcript = merge_srt(file_path)
    content_with_speaker = [
        f"{speaker}: {speech} " for idx, start, end, speaker, speech in transcript
    ]
    return "\n".join(content_with_speaker)

In [10]:
episodes = [
    "030",
    "051",
    "061",
    "069",
    "098",
    "118",
    "186",
    "240",
    "298",
    "398",
    "405",
    "430",
]

files_dir = Path("../files/bulk/rotl/")

files = [
    file
    for file in sorted(files_dir.iterdir())
    if any(episode in str(file) for episode in episodes)
]

files

[PosixPath('../files/bulk/rotl/030_-_cement_gravy_boat_of_suffering.srt'),
 PosixPath('../files/bulk/rotl/030_-_cement_gravy_boat_of_suffering.txt'),
 PosixPath('../files/bulk/rotl/051_-_in_pursuit_of_an_errant_leaf.srt'),
 PosixPath('../files/bulk/rotl/051_-_in_pursuit_of_an_errant_leaf.txt'),
 PosixPath('../files/bulk/rotl/061_-_neighborhood_stick_fight.srt'),
 PosixPath('../files/bulk/rotl/061_-_neighborhood_stick_fight.txt'),
 PosixPath('../files/bulk/rotl/069_-_campfire_spaghetti_party.srt'),
 PosixPath('../files/bulk/rotl/069_-_campfire_spaghetti_party.txt'),
 PosixPath('../files/bulk/rotl/098_-_a_mastiff_of_spinach.srt'),
 PosixPath('../files/bulk/rotl/098_-_a_mastiff_of_spinach.txt'),
 PosixPath('../files/bulk/rotl/118_-_the_articulated_hose_king.srt'),
 PosixPath('../files/bulk/rotl/118_-_the_articulated_hose_king.txt'),
 PosixPath('../files/bulk/rotl/186_-_electric_papers.srt'),
 PosixPath('../files/bulk/rotl/186_-_electric_papers.txt'),
 PosixPath('../files/bulk/rotl/240_-_w